# Generating Whisper Transcriptions

## Set Up Environment 

In [1]:
from datasets import load_dataset, Dataset, Audio
import torch
from transformers import pipeline
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import os
import pandas as pd

## Gather Data for Preprocessing

In [2]:
# This is where Audio Files are stored in my ACCRE directory, but this path may need to be changed accordingly 

folder_path = "Audio_Files/"

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
def gather_file_paths(folder_path):
    data = []
    
    # Loop through the files in the folder
    for filename in os.listdir(folder_path):
        # Ensure it's a file and has an mp3 extension
        if filename.endswith(".mp3") | filename.endswith(".wav") :
            clip_name = os.path.splitext(filename)[0]  # Get the file name without extension
            file_path = os.path.join(folder_path, filename)
            data.append({"clip": clip_name, "path": file_path})
    
    # Create a DataFrame from the gathered data
    df = pd.DataFrame(data, columns=["clip", "path"])
    
    return df

In [5]:
df = gather_file_paths(folder_path)

audio_dataset = Dataset.from_dict({"audio": df['path']}).cast_column("audio", Audio())

In [6]:
df.shape

(131, 2)

## Define stable-ts timestamps

In [7]:
#!pip install -U stable-ts

In [3]:
import os
import gc
import torch
import pandas as pd
from tqdm import tqdm
import stable_whisper
import json

In [9]:
def process_audio_batch(model, audio_files, batch_size=4):
    for i in tqdm(range(0, len(audio_files), batch_size)):
        batch = audio_files[i:i+batch_size]
        
        for j, audio_file in enumerate(batch):
            result = model.transcribe(audio_file)
            
            # Save the result as JSON
            result.save_as_json(f'stable_ts_json/stable_ts_audio_{i + j}.json')
            
            # Extract word-level data
            words_data = []
            for segment in result.segments:
                for word in segment.words:
                    words_data.append({
                        'word': word.word,
                        'start': word.start,
                        'end': word.end,
                        'probability': word.probability
                    })
            # Create DataFrame for words
            df_words = pd.DataFrame(words_data)
            # Save the DataFrame as CSV
            df_words.to_csv(f'stable_ts_csv/stable_ts_{i + j}.csv', index=False)
        
        # Clear CUDA cache
        gc.collect()
        torch.cuda.empty_cache()

In [10]:
# Load the model
model = stable_whisper.load_model('large-v3')

# Ensure the output directories exist
os.makedirs('stable_ts_json', exist_ok=True)
os.makedirs('stable_ts_csv', exist_ok=True)

audio_folder = "Audio_Files/"
audio_files = [os.path.join(audio_folder, f) for f in os.listdir(audio_folder) if f.endswith(('.mp3', '.wav'))]

In [6]:
audio_folder = "Audio_Files/"
audio_files = [os.path.join(audio_folder, f) for f in os.listdir(audio_folder) if f.endswith(('.mp3', '.wav'))]

In [17]:
import os

# Paths
AUDIO_FILES_FOLDER = 'Audio_Files'
CSV_FILES_FOLDER = 'stable_ts_csv'

# Get the list of audio file paths and extract their corresponding keys
audio_files = os.listdir(AUDIO_FILES_FOLDER)  # Ensure consistent ordering
audio_keys = [os.path.splitext(filename)[0] for filename in audio_files if (filename.endswith(".mp3") | filename.endswith(".wav")) ]

# Get the list of CSV files in the stable_ts_csv folder
csv_files = sorted( [file for file in os.listdir(CSV_FILES_FOLDER) if file.endswith("csv")] )  # Ensure consistent ordering

# Check that the number of audio files matches the number of CSV files
if len(audio_keys) != len(csv_files):
    raise ValueError("The number of audio files does not match the number of CSV files!")


In [21]:
csv_files = sorted(
    [file for file in os.listdir(CSV_FILES_FOLDER) if file.endswith("csv")],
    key=lambda x: int(x.split('_')[2].split('.')[0])  # Extract the numeric part for sorting
)

In [23]:
# Rename each CSV file to match the corresponding audio file key
for csv_file, audio_key in zip(csv_files, audio_keys):
    # Define the old and new file paths
    old_csv_path = os.path.join(CSV_FILES_FOLDER, csv_file)
    new_csv_path = os.path.join(CSV_FILES_FOLDER, f"{audio_key}.csv")
    # Rename the CSV file
    os.rename(old_csv_path, new_csv_path)

print("CSV files renamed successfully.")

CSV files renamed successfully.


In [ ]:
# Process the audio files in batches
BATCH_SIZE = 4
process_audio_batch(model, audio_files, batch_size=BATCH_SIZE)

  0%|          | 0/33 [00:00<?, ?it/s]
                                      [00:00<?, ?sec/s]
  0%|          | 0/33 [00:01<?, ?it/s]                 
Transcribe:   0%|          | 0/420.49 [00:01<?, ?sec/s]

Detected language: english



Transcribe:   7%|▋         | 28.0/420.49 [00:07<01:41,  3.88sec/s]
Transcribe:  14%|█▍        | 58.0/420.49 [00:11<01:08,  5.31sec/s]
Transcribe:  19%|█▊        | 78.0/420.49 [00:14<00:59,  5.76sec/s]
Transcribe:  26%|██▌       | 107.48/420.49 [00:17<00:44,  7.00sec/s]
Transcribe:  33%|███▎      | 137.48/420.49 [00:21<00:38,  7.27sec/s]
Transcribe:  39%|███▉      | 164.08/420.49 [00:23<00:29,  8.75sec/s]
Transcribe:  46%|████▌     | 194.08/420.49 [00:25<00:23,  9.47sec/s]
Transcribe:  53%|█████▎    | 224.08/420.49 [00:27<00:18, 10.86sec/s]
Transcribe:  60%|██████    | 254.08/420.49 [00:31<00:16,  9.95sec/s]
Transcribe:  68%|██████▊   | 284.08/420.49 [00:34<00:14,  9.67sec/s]
Transcribe:  75%|███████▍  | 314.08/420.49 [00:37<00:10,  9.87sec/s]
Transcribe:  82%|████████▏ | 344.08/420.49 [00:40<00:08,  9.54sec/s]
Transcribe:  89%|████████▉ | 374.08/420.49 [00:43<00:04,  9.81sec/s]
Transcribe:  96%|█████████▌| 404.08/420.49 [00:45<00:01, 11.48sec/s]
Transcribe: 100%|█████████▉| 420.46/420

Saved: /home/arveloic/MASCOT/stable_ts_json/stable_ts_audio_0.json



                                      [00:00<?, ?sec/s]
  0%|          | 0/33 [00:48<?, ?it/s]                 
Transcribe:   0%|          | 0/282.91 [00:00<?, ?sec/s]

...


Transcribe:   4%|▎         | 29.3/799.72 [00:04<01:51,  6.90sec/s]
Transcribe:   7%|▋         | 57.72/799.72 [00:08<01:42,  7.21sec/s]
Transcribe:  11%|█         | 87.72/799.72 [00:11<01:28,  8.09sec/s]
Transcribe:  14%|█▍        | 113.14/799.72 [00:14<01:22,  8.37sec/s]
Transcribe:  18%|█▊        | 143.14/799.72 [00:16<01:11,  9.12sec/s]
Transcribe:  22%|██▏       | 173.14/799.72 [00:20<01:10,  8.86sec/s]
Transcribe:  25%|██▌       | 203.14/799.72 [00:24<01:11,  8.35sec/s]
Transcribe:  29%|██▉       | 233.14/799.72 [00:29<01:13,  7.70sec/s]
Transcribe:  33%|███▎      | 263.14/799.72 [00:33<01:10,  7.62sec/s]
Transcribe:  37%|███▋      | 293.14/799.72 [00:36<01:01,  8.22sec/s]
Transcribe:  40%|████      | 323.14/799.72 [00:39<00:58,  8.10sec/s]
Transcribe:  44%|████▍     | 353.14/799.72 [00:42<00:51,  8.72sec/s]
Transcribe:  48%|████▊     | 383.14/799.72 [00:46<00:50,  8.30sec/s]
Transcribe:  52%|█████▏    | 413.14/799.72 [00:50<00:47,  8.18sec/s]
Transcribe:  55%|█████▌    | 443.14/7

Saved: /home/arveloic/MASCOT/stable_ts_json/stable_ts_audio_61.json



                                                    /s]
 45%|████▌     | 15/33 [1:43:59<1:59:02, 396.80s/it]   
Transcribe:   0%|          | 0/819.89 [00:00<?, ?sec/s]

Detected language: english



Transcribe:   3%|▎         | 23.0/819.89 [00:03<02:08,  6.20sec/s]
Transcribe:   6%|▌         | 51.0/819.89 [00:06<01:38,  7.80sec/s]
Transcribe:  10%|▉         | 81.0/819.89 [00:11<01:47,  6.87sec/s]
Transcribe:  14%|█▎        | 111.0/819.89 [00:15<01:35,  7.39sec/s]
Transcribe:  17%|█▋        | 141.0/819.89 [00:18<01:25,  7.93sec/s]
Transcribe:  21%|██        | 169.0/819.89 [00:23<01:30,  7.17sec/s]
Transcribe:  24%|██▍       | 198.0/819.89 [00:27<01:29,  6.92sec/s]
Transcribe:  28%|██▊       | 228.0/819.89 [00:31<01:21,  7.23sec/s]
Transcribe:  31%|███▏      | 257.0/819.89 [00:35<01:17,  7.26sec/s]
Transcribe:  35%|███▌      | 287.0/819.89 [00:39<01:10,  7.51sec/s]
Transcribe:  39%|███▊      | 316.0/819.89 [00:43<01:08,  7.36sec/s]
Transcribe:  42%|████▏     | 345.0/819.89 [00:47<01:07,  6.99sec/s]
Transcribe:  46%|████▌     | 374.0/819.89 [00:52<01:04,  6.92sec/s]
Transcribe:  49%|████▉     | 404.0/819.89 [01:04<01:35,  4.37sec/s]
Transcribe:  53%|█████▎    | 432.0/819.89 [01:09<0

In [7]:
audio_folder = "Audio_Files/"
audio_files = [os.path.join(audio_folder, f) for f in os.listdir(audio_folder) if f.endswith(('.mp3', '.wav'))]

In [8]:
audio_files[0]

'Audio_Files/008-7.mp3'

In [13]:
# Directory where your JSON files are saved
json_folder = 'stable_ts_json'

# List to hold all the data
data = []

# Loop through all JSON files in the directory
for i, filename in enumerate(os.listdir(json_folder)):
    if filename.endswith('.json'):
        clip_name = audio_files[i].split("/")[1].split(".")[0]
        file_path = os.path.join(json_folder, filename)
        
        # Open and load the JSON file
        with open(file_path, 'r') as f:
            result = json.load(f)
            
            # Check if 'segments' are in the JSON
            if 'segments' in result:
                for segment in result['segments']:
                    # Extract the required fields
                    text = segment.get('text', '')
                    start = segment.get('start', None)
                    end = segment.get('end', None)
                    
                    # Append to the data list
                    data.append({
                        'clip_name': clip_name,
                        'text': text,
                        'start': start,
                        'end': end
                    })

In [14]:
# Create a DataFrame from the data list
df = pd.DataFrame(data)

In [15]:
df.head()

,clip_name,text,start,end
0,008-7,Good morning.,2.36,2.76
1,008-7,"Today is Tuesday,",2.82,4.70
2,008-7,April 8th.,5.08,6.64
3,008-7,"Okay, you guys are doing a wonderful job sitt...",10.55,17.86
4,008-7,But we know Terry is going to be himself toge...,18.16,21.06


In [17]:
df["clip_name"].nunique()

131

In [19]:
df.to_csv("stable_ts_utterances.csv", index =  False)

## Define Whisper Pipeline

In [12]:
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=BATCH_SIZE,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
    generate_kwargs={"language": "english"},
)

def iterate_data(dataset):
    for i, item in enumerate(dataset):
        yield item["audio"]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
# 8 works fine, but larger batches could be tested 
BATCH_SIZE = 8

In [14]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=BATCH_SIZE,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
    generate_kwargs={"language": "english"},
)

In [15]:
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language=lang, task="transcribe")

In [16]:
def iterate_data(dataset):
    for i, item in enumerate(dataset):
        yield item["audio"]

In [21]:
ts_predictions = []

In [23]:
audio_dataset[-1]

{'audio': {'path': 'Audio_Files/008-12.mp3',
  'array': array([0.00228008, 0.00481903, 0.00688817, ..., 0.        , 0.        ,
         0.        ]),
  'sampling_rate': 44100}}

In [24]:
len(ts_predictions)

0

In [ ]:
# run streamed inference
for out in pipe(iterate_data(audio_dataset), 
                batch_size=BATCH_SIZE, 
                return_timestamps = True, 
               ):
    ts_predictions.append(out["chunks"])
    gc.collect()
    torch.cuda.empty_cache()

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the m

In [39]:
len(ts_predictions)

131

In [ ]:
import pickle

In [40]:
with open('whisper_largev3_transcriptions_with_timestamps.pickle', 'wb') as wlv3_transcriptions_ts:
    pickle.dump(ts_predictions, wlv3_transcriptions_ts, protocol=pickle.HIGHEST_PROTOCOL)

In [41]:
# with open('no_student_whisper_largev3_transcriptions_with_timestamps.pickle', 'wb') as wlv3_transcriptions_ts:
#     pickle.dump(ts_predictions, wlv3_transcriptions_ts, protocol=pickle.HIGHEST_PROTOCOL)

In [42]:
len(ts_predictions)

131

## Converting Transcription Output to Dataframes

In [43]:
order_of_clips = df["clip"].tolist()

In [44]:
whisper_transcription_dfs = {}
whisper_speaker_segs = {}

for transcription, clip_name in zip(ts_predictions, order_of_clips):
    whisper_transcription_df = pd.DataFrame.from_dict(transcription)
    whisper_transcription_df["start"] = whisper_transcription_df["timestamp"].apply(lambda x: x[0])
    whisper_transcription_df["end"] = whisper_transcription_df["timestamp"].apply(lambda x: x[1])
    whisper_transcription_df = whisper_transcription_df.drop("timestamp", axis=1)

    whisper_speaker_seg = whisper_transcription_df.copy()
    whisper_speaker_seg["text"] = "Teacher"
    whisper_speaker_seg = whisper_speaker_seg[["start", "end", "text"]]
    
    whisper_transcription_dfs[clip_name] = whisper_transcription_df
    whisper_speaker_segs[clip_name] = whisper_speaker_seg

In [45]:
audio_dataset[0]["audio"]

{'path': 'Audio_Files/131-9.mp3',
 'array': array([ 1.08676904e-05, -7.46108009e-04, -1.82057801e-03, ...,
         8.41914911e-08,  2.24942141e-07,  4.38241244e-07]),
 'sampling_rate': 11025}

In [46]:
from typing import Dict, List, Any

def process_transcriptions(
    whisper_transcription_dfs: Dict[str, pd.DataFrame],
    audio_dataset: List[Dict[str, Any]],
    whisper_speaker_segs: Dict[str, Any]
) -> tuple[Dict[str, pd.DataFrame], Dict[str, Any]]:
    
    # Create a dictionary for quick audio file lookup
    audio_dict = {audio["audio"]['path'].split('/')[-1].split('.')[0]: audio for audio in audio_dataset}

    for key, df in whisper_transcription_dfs.items():
        audio_file = audio_dict.get(key)
        if not audio_file:
            print(f"Warning: No matching audio file found for key {key}")
            continue

        audio_length = len(audio_file["audio"]['array']) / audio_file["audio"]['sampling_rate']
        
        # Check and update the last row
        last_index = df.index[-1]
        last_end = df.at[last_index, 'end']

        if pd.isna(last_end) or last_end > audio_length:
            df.at[last_index, 'end'] = audio_length
            whisper_speaker_segs[key][last_index, 'end'] = audio_length

    return whisper_transcription_dfs, whisper_speaker_segs

In [48]:
processed_transcription_dfs, processed_whisper_speaker_segs = process_transcriptions(whisper_transcription_dfs, audio_dataset, whisper_speaker_segs)

In [51]:
with open('whisper_largev3_transcriptions_dict.pickle', 'wb') as wm_transcriptions_dict:
    pickle.dump(processed_transcription_dfs, wm_transcriptions_dict, protocol=pickle.HIGHEST_PROTOCOL)

with open('whisper_largev3_speaker_segs_dict.pickle', 'wb') as wm_ss_dict:
    pickle.dump(processed_whisper_speaker_segs , wm_ss_dict, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('no_student_whisper_largev3_transcriptions_dict.pickle', 'wb') as wm_transcriptions_dict:
#     pickle.dump(whisper_transcription_dfs, wm_transcriptions_dict, protocol=pickle.HIGHEST_PROTOCOL)

# with open('no_student_whisper_largev3_speaker_segs_dict.pickle', 'wb') as wm_ss_dict:
#     pickle.dump(whisper_speaker_segs, wm_ss_dict, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('whisper_medium_transcriptions.pickle', 'wb') as wm_transcriptions:
#     pickle.dump(predictions, wm_transcriptions, protocol=pickle.HIGHEST_PROTOCOL)